# Attività di classificazione del testo

Come abbiamo accennato, ci concentreremo su una semplice attività di classificazione del testo basata sul dataset **AG_NEWS**, che consiste nel classificare i titoli delle notizie in una delle 4 categorie: Mondo, Sport, Economia e Scienza/Tecnologia.

## Il Dataset

Questo dataset è integrato nel modulo [`torchtext`](https://github.com/pytorch/text), quindi possiamo accedervi facilmente.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Qui, `train_dataset` e `test_dataset` contengono collezioni che restituiscono coppie di etichetta (numero di classe) e testo rispettivamente, ad esempio:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Quindi, stampiamo i primi 10 nuovi titoli dal nostro dataset:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Poiché i dataset sono iteratori, se vogliamo utilizzare i dati più volte dobbiamo convertirli in lista:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenizzazione

Ora dobbiamo convertire il testo in **numeri** che possono essere rappresentati come tensori. Se vogliamo una rappresentazione a livello di parola, dobbiamo fare due cose:
* utilizzare un **tokenizer** per suddividere il testo in **token**
* costruire un **vocabolario** di quei token.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Utilizzando il vocabolario, possiamo facilmente codificare la nostra stringa tokenizzata in un insieme di numeri:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Rappresentazione testuale Bag of Words

Poiché le parole rappresentano significati, a volte possiamo capire il senso di un testo semplicemente osservando le singole parole, indipendentemente dal loro ordine nella frase. Ad esempio, quando si classificano notizie, parole come *meteo*, *neve* probabilmente indicano *previsioni del tempo*, mentre parole come *azioni*, *dollaro* potrebbero riferirsi a *notizie finanziarie*.

La rappresentazione vettoriale **Bag of Words** (BoW) è la rappresentazione vettoriale tradizionale più comunemente utilizzata. Ogni parola è collegata a un indice del vettore, e l'elemento del vettore contiene il numero di occorrenze di una parola in un determinato documento.

![Immagine che mostra come una rappresentazione vettoriale Bag of Words sia rappresentata in memoria.](../../../../../translated_images/it/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Nota**: Puoi anche pensare al BoW come alla somma di tutti i vettori one-hot-encoded per le singole parole nel testo.

Di seguito è riportato un esempio di come generare una rappresentazione Bag of Words utilizzando la libreria python Scikit Learn:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Per calcolare il vettore bag-of-words dalla rappresentazione vettoriale del nostro dataset AG_NEWS, possiamo utilizzare la seguente funzione:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Nota:** Qui stiamo utilizzando la variabile globale `vocab_size` per specificare la dimensione predefinita del vocabolario. Poiché spesso la dimensione del vocabolario è piuttosto grande, possiamo limitarla alle parole più frequenti. Prova a ridurre il valore di `vocab_size` ed eseguire il codice qui sotto, e osserva come influisce sulla precisione. Dovresti aspettarti una diminuzione della precisione, ma non drammatica, a favore di prestazioni più elevate.


## Addestramento del classificatore BoW

Ora che abbiamo imparato a costruire la rappresentazione Bag-of-Words del nostro testo, alleniamo un classificatore su di essa. Per prima cosa, dobbiamo convertire il nostro dataset per l'addestramento in modo tale che tutte le rappresentazioni vettoriali posizionali siano trasformate in rappresentazioni Bag-of-Words. Questo può essere ottenuto passando la funzione `bowify` come parametro `collate_fn` al classico `DataLoader` di torch:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Ora definiamo una semplice rete neurale classificatrice che contiene un livello lineare. La dimensione del vettore di input è uguale a `vocab_size`, e la dimensione dell'output corrisponde al numero di classi (4). Poiché stiamo risolvendo un compito di classificazione, la funzione di attivazione finale è `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Ora definiremo il ciclo di addestramento standard di PyTorch. Poiché il nostro dataset è piuttosto grande, per il nostro scopo didattico ci alleneremo solo per un'epoca, e a volte anche per meno di un'epoca (specificare il parametro `epoch_size` ci consente di limitare l'addestramento). Riporteremo anche l'accuratezza accumulata durante l'addestramento; la frequenza di segnalazione è specificata utilizzando il parametro `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams, TriGrams e N-Grams

Una limitazione dell'approccio bag of words è che alcune parole fanno parte di espressioni composte da più parole. Ad esempio, la parola 'hot dog' ha un significato completamente diverso rispetto alle parole 'hot' e 'dog' in altri contesti. Se rappresentiamo sempre le parole 'hot' e 'dog' con gli stessi vettori, possiamo confondere il nostro modello.

Per affrontare questo problema, le **rappresentazioni N-gram** vengono spesso utilizzate nei metodi di classificazione dei documenti, dove la frequenza di ogni parola, coppia di parole o trio di parole è una caratteristica utile per addestrare i classificatori. Nella rappresentazione bigram, ad esempio, aggiungiamo tutte le coppie di parole al vocabolario, oltre alle parole originali.

Di seguito è riportato un esempio di come generare una rappresentazione bag of words bigram utilizzando Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Lo svantaggio principale dell'approccio N-gram è che la dimensione del vocabolario inizia a crescere estremamente rapidamente. In pratica, è necessario combinare la rappresentazione N-gram con alcune tecniche di riduzione della dimensionalità, come le *embedding*, che discuteremo nella prossima unità.

Per utilizzare la rappresentazione N-gram nel nostro dataset **AG News**, dobbiamo costruire un vocabolario ngram specifico:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Potremmo quindi utilizzare lo stesso codice di cui sopra per addestrare il classificatore, tuttavia, sarebbe molto inefficiente in termini di memoria. Nell'unità successiva, addestreremo un classificatore bigram utilizzando gli embeddings.

> **Nota:** Puoi mantenere solo quegli ngrammi che compaiono nel testo più di un determinato numero di volte. Questo assicurerà che i bigrammi poco frequenti vengano omessi e ridurrà significativamente la dimensionalità. Per fare ciò, imposta il parametro `min_freq` a un valore più alto e osserva come cambia la lunghezza del vocabolario.


## Frequenza Termine-Inversa Frequenza Documento TF-IDF

Nella rappresentazione BoW, le occorrenze delle parole sono ponderate in modo uniforme, indipendentemente dalla parola stessa. Tuttavia, è evidente che le parole frequenti, come *a*, *in*, ecc., sono molto meno importanti per la classificazione rispetto ai termini specializzati. Infatti, nella maggior parte dei compiti di NLP alcune parole sono più rilevanti di altre.

**TF-IDF** sta per **frequenza termine–inversa frequenza documento**. È una variazione del bag of words, dove invece di un valore binario 0/1 che indica la presenza di una parola in un documento, viene utilizzato un valore in virgola mobile, che è correlato alla frequenza di occorrenza della parola nel corpus.

Più formalmente, il peso $w_{ij}$ di una parola $i$ nel documento $j$ è definito come:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
dove
* $tf_{ij}$ è il numero di occorrenze di $i$ in $j$, ovvero il valore BoW che abbiamo visto prima
* $N$ è il numero di documenti nella collezione
* $df_i$ è il numero di documenti che contengono la parola $i$ nell'intera collezione

Il valore TF-IDF $w_{ij}$ aumenta proporzionalmente al numero di volte in cui una parola appare in un documento ed è compensato dal numero di documenti nel corpus che contengono la parola, il che aiuta a correggere il fatto che alcune parole appaiono più frequentemente di altre. Ad esempio, se la parola appare in *ogni* documento della collezione, $df_i=N$, e $w_{ij}=0$, e quei termini verrebbero completamente ignorati.

Puoi facilmente creare una vettorizzazione TF-IDF del testo utilizzando Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Conclusione

Tuttavia, anche se le rappresentazioni TF-IDF attribuiscono un peso di frequenza alle diverse parole, non sono in grado di rappresentare il significato o l'ordine. Come disse il famoso linguista J. R. Firth nel 1935: "Il significato completo di una parola è sempre contestuale, e nessuno studio del significato al di fuori del contesto può essere preso seriamente". Più avanti nel corso impareremo come catturare le informazioni contestuali dai testi utilizzando la modellazione del linguaggio.



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione AI [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un esperto umano. Non siamo responsabili per eventuali fraintendimenti o interpretazioni errate derivanti dall'uso di questa traduzione.
